In [33]:
#install if necessary
#pip install ipython-sql
#pip install pandas_bokeh

In [34]:
import numpy as np
import pandas as pd
import sqlite3
%load_ext sql

import pandas_bokeh
from bokeh.io import show,output_notebook,output_file
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import Range1d

pandas_bokeh.output_notebook()
pd.set_option("plotting.backend", "pandas_bokeh")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Loading BokehJS ...

ADD DATABASE

In [99]:
cnn = sqlite3.connect("database.db")

class Database:
    def __init__(self,csvfile,tablename):
        self.csvfile = csvfile
        self.tablename = tablename

    def add_database(self):
        df = pd.read_csv(self.csvfile)
        df.to_sql(self.tablename,cnn,if_exists='replace', index=False)
        %sql sqlite:///database.db    

data1= Database("ideal.csv","ideal")
data1.add_database()
data2= Database("train.csv","train")
data2.add_database()




CREATE DATAFRAME

In [101]:
test_data = pd.read_csv("test.csv")
ideal_data = pd.read_csv("ideal.csv")
train_data = pd.read_csv("train.csv")

INITIAL VISUALIZATION

In [102]:
class Visualization:
    def __init__(self,x,y,size,title):
        self.x = x
        self.y = y
        self.size = size
        self.title = title

    def train(self):
        visual = figure(width=280, height=250,title=self.title)
        visual.circle(self.x,self.y,size=self.size)
        return visual

class ChangeRange(Visualization):
    def __init__(self,x,y,size,title,range1,range2):
        super().__init__(x,y,size,title)
        self.range1 = range1
        self.range2 = range2
    
    def changerange(self):
        visual = figure(width=280, height=250,title=self.title)
        visual.circle(self.x,self.y,size=self.size)
        visual.y_range = Range1d(self.range1, self.range2)
        if self.range1 == self.range2:
            raise ValueError("Both of the values for y_range are the same! Please check again")
        else:
            visual.y_range = Range1d(self.range1, self.range2)
        return visual

s1 = Visualization(train_data.iloc[:,0], train_data.iloc[:,1],4,"scatter train_y1")
s2 = Visualization(train_data.iloc[:,0], train_data.iloc[:,2],4,"scatter train_y2")
s3 = ChangeRange(train_data.iloc[:,0], train_data.iloc[:,3],4,"scatter train_y3",-50,50)
s4 = Visualization(train_data.iloc[:,0], train_data.iloc[:,4],4,"scatter train_y4")
s5 = Visualization(test_data.iloc[:,0], test_data.iloc[:,1],4,"scatter test")

#show(row(s1.train(),s2.train(),s3.changerange(),s4.train(),s5.train()))


FIND Sum of Squares=∑(y -ŷ)^2 

In [103]:
def ssq(data):
    sums = []
    for num in range(1,51):
        diff = ideal_data.iloc[:,num] - data
        sum_of_square=(np.square(diff)).sum()
        sums.append([sum_of_square])
        df = pd.DataFrame(sums)
        df.index = df.index + 1
        df.columns = ["min sum of square"]
        df_sorted = df.sort_values(by="min sum of square")
    return df_sorted

ssq_df= []
for i in range(1,5):
    result = ssq(train_data.iloc[:,i])
    new_df= result[0:1].round(1)
    new_df["train func"] = "y"+str(i)
    new_df.index.name = "matching ideal func"
    new_df = new_df[["train func","min sum of square"]]
    ssq_df.append(new_df)

ssq_df = pd.concat(ssq_df)
ssq_df

,train func,min sum of square
matching ideal func,,
29,y1,30.2
40,y2,33.2
44,y3,34.4
22,y4,33.5


MATCHING DATA VISUALIZATION

In [104]:
class MatchingDataVisualization(Visualization):
    def __init__(self,x,y,size,title,trainlegend,idealx,idealy,idealsize,ideallegend,location):
        super().__init__(x,y,size,title)
        self.trainlegend = trainlegend
        self.idealx = idealx
        self.idealy = idealy
        self.idealsize = idealsize
        self.ideallegend = ideallegend
        self.location = location
    
    def matching(self):
        visual = figure(width=350, height=300,title=self.title)
        visual.circle(self.x,self.y,size=self.size,legend_label=self.trainlegend)
        visual.circle(self.idealx,self.idealy,size=self.idealsize,color="orange",legend_label=self.ideallegend)
        visual.legend.location = self.location
        return visual

class NewRange(MatchingDataVisualization):
    def __init__(self,x,y,size,title,trainlegend,idealx,idealy,idealsize,ideallegend,location,y_range1,y_range2):
        super().__init__(x,y,size,title,trainlegend,idealx,idealy,idealsize,ideallegend,location)
        self.y_range1 = y_range1
        self.y_range2 = y_range2 

    def newrange(self):
        visual = figure(width=350, height=300,title=self.title)
        visual.circle(self.x,self.y,size=self.size,legend_label=self.trainlegend)
        visual.circle(self.idealx,self.idealy,size=self.idealsize,color="orange",legend_label=self.ideallegend)
        visual.legend.location = self.location
        if self.y_range1 == self.y_range2:
            raise ValueError("Both of the values for y_range are the same! Please check again")
        else:
            visual.y_range = Range1d(self.y_range1, self.y_range2)
        return visual

v1 = MatchingDataVisualization(train_data.iloc[:,0], train_data.iloc[:,1],6,"train_y1 matches ideal_y"+str(ssq_df.index[0]),"train y_1",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[0])],2,"ideal y_"+str(ssq_df.index[0]),"top_left")
v2 = MatchingDataVisualization(train_data.iloc[:,0], train_data.iloc[:,2],6,"train_y2 matches ideal_y"+str(ssq_df.index[1]),"train y_2",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[1])],2,"ideal y_"+str(ssq_df.index[1]),"top_right")
v3 = NewRange(train_data.iloc[:,0], train_data.iloc[:,3],6,"train_y3 matches ideal_y"+str(ssq_df.index[2]),"train y_3",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[2])],2,"ideal y_"+str(ssq_df.index[2]),"top_right",-50,50)
v4 = MatchingDataVisualization(train_data.iloc[:,0], train_data.iloc[:,4],6,"train_y4 matches ideal_y"+str(ssq_df.index[3]),"train y_4",\
    ideal_data.iloc[:,0],ideal_data["y"+str(ssq_df.index[3])],2,"ideal y_"+str(ssq_df.index[3]),"top_center")

#show(row(v1.matching(),v2.matching(),v3.newrange(),v4.matching()))
        

TEST DATA MAPPING

In [105]:
new_test_df= []
for xva in test_data.iloc[:,0]:
    values = ideal_data.loc[ideal_data.iloc[:,0] == xva]
    new_test_df.append(values)

new_test_df = pd.concat(new_test_df, ignore_index=True)
new_test_df.insert (1, "y", test_data.iloc[:,1])
selected_df_one = new_test_df[["x","y","y"+str(ssq_df.index[0]),"y"+str(ssq_df.index[1]),"y"+str(ssq_df.index[2]),"y"+str(ssq_df.index[3])]]


selected_df = selected_df_one.copy()

selected_df["diff1"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,2])
selected_df["diff2"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,3])
selected_df["diff3"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,4])
selected_df["diff4"] = abs(selected_df.iloc[:,1] - selected_df.iloc[:,5])

selected_df["No. of ideal func"] = ""

In [106]:
a=(abs(train_data.iloc[:,1]-ideal_data["y"+str(ssq_df.index[0])])).max()*(np.sqrt(2))
b=(abs(train_data.iloc[:,2]-ideal_data["y"+str(ssq_df.index[1])])).max()*(np.sqrt(2))
c=(abs(train_data.iloc[:,3]-ideal_data["y"+str(ssq_df.index[2])])).max()*(np.sqrt(2))
d=(abs(train_data.iloc[:,4]-ideal_data["y"+str(ssq_df.index[3])])).max()*(np.sqrt(2))

class Map:
    def __init__(self,data,criterion,funcname,delta):
        self.data = data
        self.criterion = criterion
        self.funcname = funcname
        self.delta = delta

    def mapping(self):
        required_df=self.data.loc[self.data.iloc[:,2] < self.criterion]
        required_df_copy=required_df.copy()
        required_df_copy["No. of ideal func"] = self.funcname
        rename = required_df_copy.rename(columns = {self.delta : "Delta"})
        return rename

ymap1 = Map(selected_df.iloc[:, [0,1,6,10]],a,"y"+str(ssq_df.index[0]),"diff1")
ymap2 = Map(selected_df.iloc[:, [0,1,7,10]],b,"y"+str(ssq_df.index[1]),"diff2")
ymap3 = Map(selected_df.iloc[:, [0,1,8,10]],c,"y"+str(ssq_df.index[2]),"diff3")
ymap4 = Map(selected_df.iloc[:, [0,1,9,10]],d,"y"+str(ssq_df.index[3]),"diff4")

frames = [ymap1.mapping(),ymap2.mapping(),ymap3.mapping(),ymap4.mapping()]
mapped_data = pd.concat(frames)
mapped_data.sort_index(inplace=True)

,x,y,Delta,No. of ideal func
1,8.6,636.259640,0.203640,y22
2,-12.9,-1980.184200,0.094800,y29
3,-9.5,-767.691960,0.566960,y29
6,-9.6,385.555880,0.411200,y40
8,10.2,0.546554,0.207712,y40
8,10.2,0.546554,0.557690,y44
9,16.4,42.319393,0.589447,y40
12,-4.3,80.029010,0.522010,y22
14,14.2,2862.828400,0.459600,y22
18,-12.9,-0.116593,0.107735,y44


EXPORT TO DATABASE

In [107]:
mapped_data.to_sql("mapped_test",cnn,if_exists='replace', index=False)

32

In [108]:
t1 = figure(width=600, height=400,title="mapped data points")
t1.circle(test_data.iloc[:,0], test_data.iloc[:,1], size=4,legend_label="test_data")
t1.circle(ymap1.mapping().iloc[:,0], ymap1.mapping().iloc[:,1], size=6,color="red",legend_label="y"+str(ssq_df.index[0]))
t1.circle(ymap2.mapping().iloc[:,0], ymap2.mapping().iloc[:,1], size=6,color="orange",legend_label="y"+str(ssq_df.index[1]))
t1.circle(ymap3.mapping().iloc[:,0], ymap3.mapping().iloc[:,1], size=6,color="purple",legend_label="y"+str(ssq_df.index[2]))
t1.circle(ymap4.mapping().iloc[:,0], ymap4.mapping().iloc[:,1], size=6,color="black",legend_label="y"+str(ssq_df.index[3]))
t1.legend.location = "bottom_right"
#show(t1)

